This notebook contains the code to upload documents to an Oracle DB and query the DB to retrieve the top-k closest vectors to a given query vector.

In [ ]:
!pip install openai
!pip install sentence-transformers
!pip install oracledb
!pip install numpy
!pip install llama-index==0.9.39
!pip install oracledb
!pip install pypdf


Import the required libraries

In [6]:
import array
import os
import time

import openai

from llama_index.readers import SimpleDirectoryReader

from typing import List
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import re
import numpy as np
from llama_index.schema import TextNode, BaseNode
import json
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)

# to generate id from text
import hashlib

from llama_index.schema import TextNode, BaseNode

import oracledb
import logging


Define the configuration parameters and create a connection pool to the Oracle DB.

In [7]:
# The username for your local Oracle DB
DB_USER = "hr"

# The password for your local Oracle DB
DB_PWD = "oracle"

# The IP address of your local Oracle DB. For a local DB, this is often "localhost"
DB_HOST_IP = "127.0.0.1"

# The service name of your local Oracle DB
DB_SERVICE = "freepdb1"

DSN = f"{DB_HOST_IP}/{DB_SERVICE}"

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
MAX_CHUNK_SIZE = 400
CHUNK_OVERLAP = 50
# for retrieval
TOP_K = 2
# must be aligned with the create_tables.sql used
EMBEDDINGS_BITS = 64

API_KEY = "sk-"
#pool = oracledb.create_pool(user=DB_USER, password=DB_PWD, dsn=DSN, min=1, max=10, increment=1)

This section contains the code to upload documents to the Oracle DB. The documents are split into chunks, and the embeddings for each chunk are generated using a pre-trained Sentence Transformer model. The embeddings are then saved to the DB. The code also contains a function to retrieve the top-k closest vectors to a given query vector.

In [8]:
def upload_documents(user_id, files):
    print("Starting document upload process for user:", user_id)
    # Process each file
    for file in files:
        document_name = os.path.basename(file)
        print("Processing file:", document_name)
        pages_text, _ = read_and_split_in_pages([file])
        model = SentenceTransformer(EMBED_MODEL)
        embeddings = model.encode(pages_text, convert_to_tensor=False, show_progress_bar=True)
        save_embeddings(user_id, document_name, pages_text, embeddings)
    return "Documents uploaded successfully!"


def save_embeddings(user_id, document_name, pages_text, embeddings):
    print("Saving embeddings for document:", document_name)
    try:
        connection = oracledb.connect(user=DB_USER, password=DB_PWD, dsn=DSN)
        cursor = connection.cursor()

        # Generate document ID and insert into BOOKS table
        document_id = int(hashlib.sha256(document_name.encode()).hexdigest(), 16) % (
                10 ** 8)  # Generate a unique integer ID
        insert_document_sql = "INSERT INTO BOOKS (ID, NAME, USERID) VALUES (:1, :2, :3)"
        print("Executing SQL:", insert_document_sql, "with params:", (document_id, document_name, user_id))
        cursor.execute(insert_document_sql, (document_id, document_name, user_id))

        for page_num, (chunk_text, embedding) in enumerate(zip(pages_text, embeddings), start=1):
            # Generate chunk ID
            chunk_id = hashlib.sha256(f"{document_id}-{page_num}".encode()).hexdigest()

            # Insert chunk into CHUNKS table
            insert_chunk_sql = "INSERT INTO CHUNKS (ID, CHUNK, PAGE_NUM, BOOK_ID) VALUES (:1, :2, :3, :4)"
            #print("Executing SQL:", insert_chunk_sql, "with params:", (chunk_id, chunk_text, str(page_num), document_id))
            cursor.execute(insert_chunk_sql, (chunk_id, chunk_text, str(page_num), document_id))

            # Convert embedding to JSON string
            embedding = np.array(embedding, dtype=np.float64)
            embedding_json = json.dumps(embedding.tolist())

            # Insert embedding into VECTORS table
            insert_vector_sql = "INSERT INTO VECTORS (ID, VEC) VALUES (:1, :2)"
            #print("Executing SQL:", insert_vector_sql, "with params:", (chunk_id, embedding_json))
            cursor.execute(insert_vector_sql, (chunk_id, embedding_json))

        connection.commit()
        cursor.close()
        connection.close()
    except oracledb.DatabaseError as e:
        print(f"Error occurred: {e}")


def read_and_split_in_pages(input_files):
    print("Reading and splitting pages for input files")
    pages = SimpleDirectoryReader(input_files=input_files).load_data()
    for doc in pages:
        doc.text = preprocess_text(doc.text)
    pages = remove_short_pages(pages, threshold=10)
    pages_text = [doc.text for doc in pages]
    pages_num = [doc.metadata["page_label"] for doc in pages]
    return pages_text, pages_num


def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    return text.strip()


def remove_short_pages(pages, threshold):
    return [doc for doc in pages if len(doc.text.split()) > threshold]

In [9]:
def clear_db_sessions():
    try:
        connection = oracledb.connect(user="system", password="oracle", dsn="localhost:1521/freepdb1")
        cursor = connection.cursor()

        # Get all active sessions
        cursor.execute("SELECT sid, serial# FROM v$session WHERE username = 'HR' OR username IS NULL")
        sessions = cursor.fetchall()
        print(f"Sessions to kill: {sessions}")

        # Kill each session
        for sid, serial in sessions:
            cursor.execute(f"ALTER SYSTEM KILL SESSION '{sid},{serial}'")

        connection.commit()
    except oracledb.DatabaseError as e:
        print(f"Error occurred: {e}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

Test the document upload process. Upload documents for a business user and a technical user.

In [10]:
clear_db_sessions()

business_user_id = "business_user"
business_user_files = [
    "c:\\tmp\\digital-transformation.pdf",
    "c:\\tmp\\change-management.pdf"
]

technical_user_id = "technical_user"
technical_user_files = [
    "c:\\tmp\\blockchain_paper.pdf",
    "c:\\tmp\\microservices.pdf"
]

upload_documents(business_user_id, business_user_files)
upload_documents(technical_user_id, technical_user_files)


Sessions to kill: [(1, 25683), (2, 14420), (3, 52488), (4, 22216), (5, 54178), (6, 26479), (7, 62053), (8, 54796), (9, 32726), (10, 14867), (11, 49275), (12, 16749), (13, 12606), (14, 52317), (15, 26191), (16, 13981), (17, 8126), (19, 34884), (20, 24461), (21, 26842), (22, 1658), (23, 34679), (24, 32449), (25, 51624), (26, 63799), (27, 17757), (34, 22070), (37, 3305), (40, 53673), (41, 33139), (47, 24670), (163, 5398), (164, 25875), (165, 61154), (166, 46013), (167, 18140), (169, 61026), (170, 46948), (171, 51534), (172, 32921), (173, 49050), (174, 43650), (175, 29586), (177, 32542), (179, 38961), (180, 3471), (181, 8481), (182, 13850), (183, 18720), (184, 2675), (185, 58967), (186, 50056), (187, 37596), (188, 11425), (190, 24539), (193, 43890), (194, 51613), (195, 44199), (196, 50318), (204, 64415), (207, 22735), (210, 29241)]
Error occurred: ORA-00026: missing or invalid session ID
Help: https://docs.oracle.com/error-help/db/ora-00026/
Starting document upload process for user: busin

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Saving embeddings for document: digital-transformation.pdf
Executing SQL: INSERT INTO BOOKS (ID, NAME, USERID) VALUES (:1, :2, :3) with params: (38837196, 'digital-transformation.pdf', 'business_user')
Processing file: change-management.pdf
Reading and splitting pages for input files


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saving embeddings for document: change-management.pdf
Executing SQL: INSERT INTO BOOKS (ID, NAME, USERID) VALUES (:1, :2, :3) with params: (18008419, 'change-management.pdf', 'business_user')
Starting document upload process for user: technical_user
Processing file: blockchain_paper.pdf
Reading and splitting pages for input files


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saving embeddings for document: blockchain_paper.pdf
Executing SQL: INSERT INTO BOOKS (ID, NAME, USERID) VALUES (:1, :2, :3) with params: (61788562, 'blockchain_paper.pdf', 'technical_user')
Processing file: microservices.pdf
Reading and splitting pages for input files


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Saving embeddings for document: microservices.pdf
Executing SQL: INSERT INTO BOOKS (ID, NAME, USERID) VALUES (:1, :2, :3) with params: (21024628, 'microservices.pdf', 'technical_user')


'Documents uploaded successfully!'

Define the function to query the Oracle DB to retrieve the top-k closest vectors to a given query vector. The function uses the cosine distance to compute the similarity between vectors.

In [11]:
def oracle_query_cosine(embed_query: List[float], user_id, top_k: int = 2, verbose=False):
    """
    Executes a query against an Oracle database to find the top_k closest vectors to the given embedding.

    Args:
        embed_query (List[float]): A list of floats representing the query vector embedding.
        top_k (int, optional): The number of closest vectors to retrieve. Defaults to 2.
        verbose (bool, optional): If set to True, additional information about the query and execution time will be printed. Defaults to False.

    Returns:
        VectorStoreQueryResult: Object containing the query results, including nodes, similarities, and ids.
        :param verbose:
        :param top_k:
        :param embed_query:
        :param user_id:
    """
    start_time = time.time()

    try:

        connection = oracledb.connect(user=DB_USER, password=DB_PWD, dsn=DSN)
        cursor = connection.cursor()
        # 'f' single precision 'd' double precision
        #array_type = "d" if EMBEDDINGS_BITS == 64 else "f"
        array_query = array.array('f', embed_query)

        select = f"""select V.id, C.CHUNK, C.PAGE_NUM, 
                    cosine_distance(V.VEC, :1) as d,
                    B.NAME 
                    from VECTORS V, CHUNKS C, BOOKS B
                    where C.ID = V.ID and
                    C.BOOK_ID = B.ID and
                    B.USERID = '{user_id}'
                    order by d
                    FETCH FIRST {top_k} ROWS ONLY"""

        if verbose:
            logging.info(f"SQL Query 2: {select}")

        print(f"Array query 2: {array_query}")
        cursor.execute(select, [array_query])
        rows = cursor.fetchall()
        print(f"Rows: {rows}")

        result_nodes, node_ids, similarities = [], [], []
        print(f"Result nodes: {result_nodes}")

        # prepare output
        for row in rows:
            # row[1] is a clob
            full_clob_data = row[1].read()
            result_nodes.append(
                TextNode(
                    id_=row[0],
                    text=full_clob_data,
                    metadata={"file_name": row[4], "page_label": row[2]},
                )
            )
            node_ids.append(row[0])
            similarities.append(row[3])

        q_result = VectorStoreQueryResult(
            nodes=result_nodes, similarities=similarities, ids=node_ids
        )

        elapsed_time = time.time() - start_time

        if verbose:
            logging.info(f"Query duration: {round(elapsed_time, 1)} sec.")

        return q_result
        
    except Exception as e:
        logging.error(f"Error occurred in oracle_query 2: {e}")
        return None



Test the query function by querying the Oracle DB for the top-k closest vectors to a given query vector. The query vector is generated using a pre-trained Sentence Transformer model. The function retrieves the closest vectors for a business user and a technical user. The retrieved vectors are then used to generate an answer to a question using a pre-trained GPT-4 model. The answer is appended to the chat history. The chat history is then displayed.

In [20]:
# Global variable to store chat history
global chat_history
chat_history = []


def generate_answer(question, context):
    # Append the new question to the chat history
    #global chat_history
    chat_history.append({"role": "user", "content": question})

    openai.api_key = API_KEY
    client = OpenAI(
        api_key=openai.api_key,
    )
    stream = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user",
                   "content": f"Answer the following question based on the provided text and describe the metadata that it came from, if the answer can't be concluded from the text just say that the provided text is insufficient to provide an answer, don't respond from your own knowledge. The text is:.\n\nContext: {context}\n\nQuestion: {question}"}],
        stream=True,
    )
    print(
        f"Answer the following question based on the provided text and describe the metadata that it came from, if the text doesn't contain the answer just say that the provided text is insufficient to provide an answer, don't respond from your own knowledge. The text is:.\n\nContext: {context}\n\nQuestion: {question}")
    collected_chunks = []
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            collected_chunks.append(chunk.choices[0].delta.content)

    llm_answer = ''.join(collected_chunks).strip()
    print("LLM Answer:", llm_answer)

    # Append the generated answer to the chat history
    chat_history.append({"role": "assistant", "content": llm_answer})

    return llm_answer


def chat(user_id, question):
    global chat_history  # Declare chat_history as a global variable

    context = test_oracle_query(user_id, question)
    print("Retrieved context:", context)
    answer = generate_answer(question, context)
    print("Generated answer:", answer)

    # Convert the chat history to a string format suitable for display
    chat_history_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in chat_history])
    #print("Chat history:", chat_history_str)

    return chat_history_str


def test_oracle_query(user_id, question):
    top_k = 5
    verbose = True
    print("Retrieving documents for user:", user_id)
    print("Question:", question)
    model = SentenceTransformer(EMBED_MODEL)
    test_question_emeddings = model.encode(question, convert_to_tensor=False, show_progress_bar=True)

    testresults2 = oracle_query_cosine(test_question_emeddings.tolist(), user_id, top_k, verbose)
    print(f"Test results 2: {testresults2}")

    context = ""
    for result in testresults2.nodes:
        chunk_text = result.text
        chunk_metadata = result.metadata
        context += chunk_text + " " + str(chunk_metadata) + " "

    return context.strip()

In [21]:
chat_user_id = "technical_user"
chat_question = "what is the potential of using blockchain with BIM?"

print(chat(chat_user_id, chat_question))



Retrieving documents for user: technical_user
Question: what is the potential of using blockchain with BIM?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Array query 2: array('f', [-0.015785766765475273, 0.0009981109760701656, -0.048373933881521225, -0.016667965799570084, -0.062468912452459335, -0.02723805420100689, -0.004929180257022381, 0.04940257593989372, 0.002238634740933776, -0.02024371363222599, -0.019948285073041916, 0.04359608516097069, 0.02103298343718052, -0.000399686919990927, 0.09506251662969589, 0.11448220163583755, 0.09343384951353073, 0.023033209145069122, 0.06215339154005051, 0.0269812922924757, -0.036601148545742035, -0.061342913657426834, -0.04309963807463646, 0.02289023995399475, -0.054171621799468994, -0.04470172896981239, 0.035397183150053024, 0.075709268450737, -0.05186079442501068, -0.05832734704017639, 0.02747039683163166, 0.12075915932655334, 0.05966165289282799, 0.0019234840292483568, -0.1816176325082779, 0.02609996870160103, 0.05955231934785843, -0.026852596551179886, -0.005764928180724382, -0.0633297711610794, 0.0054460326209664345, 0.010879441164433956, 0.03729305416345596, -0.1166454553604126, 0.0053397356